In [1]:
# pip install snscrape
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [2]:
neighborhoods = {
    'suroeste': ["4 de Marzo", "12 de Mayo", "25 de Julio", "Abel Gilbert", "Antonio Neumane",
                 "Barrio Garay", "Barrio Lindo", "Barrio Puerto Lisa", "Bastión del Suburbio",
                 "Batallón del Suburbio", "Bellavista", "Cdla. Atahualpa", "Coop. 9 de Julio",
                 "Coop. Ángel Duarte", "Cooperativa El Paraíso", "Coop. Empleados del Municipio",
                 "Coop. Jacobo Bucaram", "Coop. Jaime Toral", "Coop. Mandela", "Coop. Monseñor Leonidas Proaño",
                 "Coop. Santiago de Guayaquil", "El Cisne", "Estero Salado", "Fe y Alegría", "Febres Cordero",
                 "García Moreno", "Guangala", "Independencia", "Isla Trinitaria", "Jardines del Salado",
                 "La Chala", "La Ferroviaria", "La Fuente", "La Fuerza de los Pobres", "Letamendi", "Los Ángeles",
                 "Luz de Guayas", "Puerto Azul Sur", "San Pedro", "Sol Naciente", "Urbanización Girasol"],
    
    'centro': ["9 de Octubre Oeste", "Barrio Orellana", "Bolívar", "Cerro del Carmen",
               "Cerro Santa Ana", "La Bahía", "Las Peñas", "Malecón 2000", "Malecón del Salado",
               "Olmedo", "Pedro Carbo", "Roca", "Rocafuerte", "Sucre", "Urdaneta"]
}

In [3]:
crimes = ['asesinato', 'violación', 'robo', 'homicidio', 'asalto', '(violencia doméstica)', 
         'secuestro', '(abuso sexual)', 'falsificación', 'sicariato', 'drogas', 'femicidio',
         'armas', 'sicario', 'asesinaron', 'violaron', 'robaron', 'asaltaron', 'secuestraron',
         'falsificaron', 'amedrentaron', 'amenazaron']

In [4]:
def build_query(neighborhood, crimes, i_date, f_date):
    neighborhood = '(' + neighborhood + ')'
    date_range = f'since:{i_date} until:{f_date}'
    union_crimes = '(' + ' OR '.join(crimes) + ')'
    lang = 'lang:es'
    no_replies = '-filter:replies'
    query = f'{neighborhood.lower()} {union_crimes} {date_range} {lang} {no_replies}'
    
    return query

In [5]:
#query_test = build_query('Alborada', crimes, '2018-01-01', '2022-11-10')
#query_test

In [6]:
def get_tweets(query, max_tweets):
    l_tweets = []

    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
        if i == max_tweets:
            break
        l_tweets.append([
            tweet.date, 
            tweet.content,
            tweet.url,
            tweet.likeCount,
            tweet.lang,
            tweet.retweetedTweet,
            tweet.user.username,
            tweet.id])
        
    return l_tweets

In [7]:
#l_tweets_test = get_tweets(query_test, 2)
#l_tweets_test

In [8]:
def create_df(l_tweets):
    df = pd.DataFrame(l_tweets, columns=['date',
                                         'content', 
                                         'url',
                                         'like_count',
                                         'language', 
                                         'is_retweeted', 
                                         'username',
                                         'id'])
    return df

In [9]:
# df_test = create_df(l_tweets_test)
# df_test.head()

In [10]:
for sector, l_neighborhoods in neighborhoods.items():
    df_sector = create_df([])
    for neighborhood in l_neighborhoods:
        query = build_query(neighborhood, crimes, i_date='2018-01-01', f_date='2022-11-10')
        l_tweets = get_tweets(query, 15_000)
        df_neighborhood = create_df(l_tweets)
        df_neighborhood["sector"] = sector
        df_neighborhood["neighborhood"] = neighborhood
        
        df_sector = pd.concat([df_sector, df_neighborhood])
    
    df_sector.reset_index(inplace=True, drop=True)
    df_sector.to_csv(f'../data/tweets_{sector}.csv', index=False)

C:\Users\erick\AppData\Local\Temp\ipykernel_47392\2134493696.py:9: FutureWarning: content is deprecated, use rawContent instead
  tweet.content,
C:\Users\erick\AppData\Local\Temp\ipykernel_47392\2134493696.py:9: FutureWarning: content is deprecated, use rawContent instead
  tweet.content,
C:\Users\erick\AppData\Local\Temp\ipykernel_47392\2134493696.py:9: FutureWarning: content is deprecated, use rawContent instead
  tweet.content,
C:\Users\erick\AppData\Local\Temp\ipykernel_47392\2134493696.py:9: FutureWarning: content is deprecated, use rawContent instead
  tweet.content,
C:\Users\erick\AppData\Local\Temp\ipykernel_47392\2134493696.py:9: FutureWarning: content is deprecated, use rawContent instead
  tweet.content,
C:\Users\erick\AppData\Local\Temp\ipykernel_47392\2134493696.py:9: FutureWarning: content is deprecated, use rawContent instead
  tweet.content,
C:\Users\erick\AppData\Local\Temp\ipykernel_47392\2134493696.py:9: FutureWarning: content is deprecated, use rawContent instead
  

In [11]:
df_suroeste = pd.read_csv('../data/tweets_suroeste.csv')

In [12]:
df_suroeste.shape

(75279, 10)

In [13]:
df_suroeste.tail()

,date,content,url,like_count,language,is_retweeted,username,id,sector,neighborhood
75274,2018-01-12 00:18:35+00:00,La violación de Nankín: el genocidio del Imper...,https://twitter.com/Alma1205/status/9516093140...,0,es,NaN,Alma1205,951609314082459648,suroeste,Sol Naciente
75275,2018-01-11 23:36:07+00:00,La violación de Nankín: el genocidio del Imper...,https://twitter.com/thewildzing/status/9515986...,0,es,NaN,thewildzing,951598628602195968,suroeste,Sol Naciente
75276,2018-01-03 13:40:14+00:00,Robaron dos motos en una Comisaría - El hecho ...,https://twitter.com/NoticiasPunilla/status/948...,0,es,NaN,NoticiasPunilla,948549566592372741,suroeste,Sol Naciente
75277,2018-01-03 11:37:39+00:00,El primer robo del 2018 en Córdoba ocurrió en ...,https://twitter.com/ElDoce/status/948518716945...,7,es,NaN,ElDoce,948518716945387522,suroeste,Sol Naciente
75278,2022-06-30 00:24:24+00:00,#URGENTE\n\nEn estos momentos se reporta un si...,https://twitter.com/PulsoProEc/status/15423029...,5,es,NaN,PulsoProEc,1542302998365356039,suroeste,Urbanización Girasol


In [14]:
df_centro = pd.read_csv('../data/tweets_centro.csv')

In [15]:
df_centro.shape

(64119, 10)

In [16]:
df_centro.head()

,date,content,url,like_count,language,is_retweeted,username,id,sector,neighborhood
0,2019-11-30 16:42:04+00:00,ÚLTIMA HORA | Fiscalía Regional de Panamá Oest...,https://twitter.com/UltimaHoraPma/status/12008...,0,es,NaN,UltimaHoraPma,1200817281787412482,centro,9 de Octubre Oeste
1,2022-10-21 03:45:06+00:00,"1/2 Ayer 19/10/2022, se escucharon disparos al...",https://twitter.com/v4li3nte/status/1583303337...,39,es,NaN,v4li3nte,1583303337855705089,centro,Barrio Orellana
2,2022-10-20 00:01:38+00:00,#Ecuador | Grave: Velorio con bala\n\nEl asesi...,https://twitter.com/AntisanaNews/status/158288...,4,es,NaN,AntisanaNews,1582884710111653888,centro,Barrio Orellana
3,2022-09-08 20:01:59+00:00,"Capturado por el Cicpc en el barrio Mesuca, de...",https://twitter.com/RCamachoVzla/status/156796...,262,es,NaN,RCamachoVzla,1567966500652457986,centro,Barrio Orellana
4,2022-07-18 00:42:27+00:00,Acabo de recibir una llamada de un pana para d...,https://twitter.com/Ey_orellana/status/1548830...,5,es,NaN,Ey_orellana,1548830523899904005,centro,Barrio Orellana
